In [12]:
import numpy as np

# Get inputs

In [13]:
# with open("test_input.txt") as f:
with open("input.txt") as f:
    inp = f.readlines()
    inp = [line.rstrip() for line in inp]

In [14]:
cleaned = []
for line in inp:
    state, coords = line.split()
    x,y,z = coords.split(',')
    x = x[2:].split('..')
    y = y[2:].split('..')
    z = z[2:].split('..')
    
    cleaned.append([state, [(int(x[0]), int(x[1])+1), (int(y[0]), int(y[1])+1), (int(z[0]), int(z[1])+1)]])

In [15]:
cleaned

[['on', [(-45, 8), (-17, 28), (5, 50)]],
 ['on', [(-47, 7), (-17, 31), (-24, 27)]],
 ['on', [(-41, 4), (-7, 40), (-40, 14)]],
 ['on', [(-7, 38), (-13, 40), (-8, 43)]],
 ['on', [(-28, 26), (1, 48), (-30, 17)]],
 ['on', [(-17, 36), (-41, 9), (-24, 29)]],
 ['on', [(-12, 41), (-16, 37), (-9, 39)]],
 ['on', [(-42, 6), (-43, 6), (-41, 6)]],
 ['on', [(-41, 9), (-19, 33), (-5, 49)]],
 ['on', [(-7, 46), (-48, -3), (-36, 19)]],
 ['off', [(-30, -16), (-18, -4), (14, 32)]],
 ['on', [(-34, 12), (-2, 44), (-40, 13)]],
 ['off', [(15, 32), (8, 24), (-25, -13)]],
 ['on', [(1, 50), (-7, 40), (-21, 28)]],
 ['off', [(-10, 1), (24, 34), (-25, -7)]],
 ['on', [(-45, 2), (-5, 46), (-20, 32)]],
 ['off', [(-45, -27), (-3, 16), (29, 44)]],
 ['on', [(-28, 23), (-13, 34), (-17, 32)]],
 ['off', [(40, 50), (9, 25), (-38, -20)]],
 ['on', [(-31, 21), (-21, 32), (-10, 41)]],
 ['on', [(58139, 70016), (-49609, -18905), (19361, 36257)]],
 ['on', [(-39287, -12308), (66131, 82073), (13276, 35311)]],
 ['on', [(-38949, -17452

## Q1

In [16]:
np.add((3,3),50)

array([53, 53])

In [17]:
reactor = np.full((101,101,101), 0,dtype=np.int8)

In [18]:
def run(reactor, cleaned):
    n = 50
    for state, coords in cleaned:
        x,y,z = coords
        x = np.add(x,n)
        y = np.add(y,n)
        z = np.add(z,n)

        print(f"{state}, {x, y, z}")
        state = 1 if 'on' in state else 0
        
        reactor[x[0]:x[1], y[0]:y[1], z[0]:z[1]] = state
    
        print(f"{np.sum(reactor)}")
    
    return reactor

In [19]:
np.sum(run(reactor, cleaned))

on, (array([ 5, 58]), array([33, 78]), array([ 55, 100]))
107325
on, (array([ 3, 57]), array([33, 81]), array([26, 77]))
188037
on, (array([ 9, 54]), array([43, 90]), array([10, 64]))
237267
on, (array([43, 88]), array([37, 90]), array([42, 93]))
324422
on, (array([22, 76]), array([51, 98]), array([20, 67]))
362729
on, (array([33, 86]), array([ 9, 59]), array([26, 79]))
443973
on, (array([38, 91]), array([34, 87]), array([41, 89]))
453953
on, (array([ 8, 56]), array([ 7, 56]), array([ 9, 56]))
505397
on, (array([ 9, 59]), array([31, 83]), array([45, 99]))
513375
on, (array([43, 96]), array([ 2, 47]), array([14, 69]))
565448
off, (array([20, 34]), array([32, 46]), array([64, 82]))
561920
on, (array([16, 62]), array([48, 94]), array([10, 63]))
568100
off, (array([65, 82]), array([58, 74]), array([25, 37]))
565922
on, (array([ 51, 100]), array([43, 90]), array([29, 78]))
593974
off, (array([40, 51]), array([74, 84]), array([25, 43]))
591994
on, (array([ 5, 52]), array([45, 96]), array([30

607657

## Q2